In [1]:
import os
import pickle
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
from tqdm import tqdm
import librosa
import math
import random
import pandas as pd
import IPython.display
import librosa.display
ms.use('seaborn-muted')
%matplotlib inline

In [2]:
data_dir = '/home/rashmi/Dataset/pre-processed-chromagram/'
labels_df_path = '{}df_iemocap.csv'.format(data_dir)
audio_vectors_path = '{}audio_vectors_5.pkl'.format(data_dir)
hop_length = 512
sr=44100

In [3]:
labels_df = pd.read_csv(labels_df_path)
audio_vectors = pickle.load(open(audio_vectors_path, 'rb'))

In [4]:
columns = ['wav_file', 'label', 'sig_mean', 'sig_std', 'rmse_mean', 'rmse_std', 'silence', 'harmonic', 'auto_corr_max', 'auto_corr_std','chroma1','chroma2','chroma3','chroma4','chroma5','chroma6','chroma7','chroma8','chroma9','chroma10','chroma11','chroma12']
df_features = pd.DataFrame(columns=columns)

In [5]:
emotion_dict = {'ang': 0,
                'hap': 1,
                'exc': 2,
                'sad': 3,
                'fru': 4,
                'fea': 5,
                'sur': 6,
                'neu': 7,
                'xxx': 8,
                'oth': 8}

data_dir = '/home/rashmi/Dataset/pre-processed-chromagram/'
labels_path = '{}df_iemocap.csv'.format(data_dir)
audio_vectors_path = '{}audio_vectors_'.format(data_dir)

In [8]:
labels_df = pd.read_csv(labels_path)

for sess in [5]:
        audio_vectors = pickle.load(open('{}{}.pkl'.format(audio_vectors_path, sess), 'rb'))
        for index, row in tqdm(labels_df[labels_df['wav_file'].str.contains('Ses0{}'.format(sess))].iterrows()):
            try:
                wav_file_name = row['wav_file']
                label = emotion_dict[row['emotion']]
                y = audio_vectors[wav_file_name]

                feature_list = [wav_file_name, label]  # wav_file, label
                sig_mean = np.mean(abs(y))
                feature_list.append(sig_mean)  # sig_mean
                feature_list.append(np.std(y))  # sig_std

                rmse = librosa.feature.rms(y + 0.0001)[0]
                feature_list.append(np.mean(rmse))  # rmse_mean
                feature_list.append(np.std(rmse))  # rmse_std

                silence = 0
                for e in rmse:
                    if e <= 0.4 * np.mean(rmse):
                        silence += 1
                silence /= float(len(rmse))
                feature_list.append(silence)  # silence

                y_harmonic = librosa.effects.hpss(y)[0]
                feature_list.append(np.mean(y_harmonic) * 1000)  # harmonic (scaled by 1000)

                # based on the pitch detection algorithm mentioned here:
                # http://access.feld.cvut.cz/view.php?cisloclanku=2009060001
                cl = 0.45 * sig_mean
                center_clipped = []
                for s in y:
                    if s >= cl:
                        center_clipped.append(s - cl)
                    elif s <= -cl:
                        center_clipped.append(s + cl)
                    elif np.abs(s) < cl:
                        center_clipped.append(0)
                auto_corrs = librosa.core.autocorrelate(np.array(center_clipped))
                feature_list.append(1000 * np.max(auto_corrs)/len(auto_corrs))  # auto_corr_max (scaled by 1000)
                feature_list.append(np.std(auto_corrs))  # auto_corr_std
                
                
                #chromagram
                stft=np.abs(librosa.stft(y))
                chroma_stft=np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T,axis=0)

                for i in range(0,12):
                    feature_list.append(chroma[i])  # chroma
                

                df_features = df_features.append(pd.DataFrame(feature_list, index=columns).transpose(), ignore_index=True)
            except:
                print('Some exception occured')

df_features.to_csv('/home/rashmi/Dataset/pre-processed-chromagram/audio_features.csv', index=False)

      start_time  end_time                wav_file emotion  val  act  dom
0           5.18      9.76     Ses05F_impro04_F000     sad  2.5  2.0  3.0
1          15.79     26.20     Ses05F_impro04_F001     xxx  2.0  3.0  3.5
2          29.45     36.38     Ses05F_impro04_F002     fru  2.0  3.0  3.0
3          40.14     44.28     Ses05F_impro04_F003     fru  1.5  3.5  3.5
4          49.54     66.46     Ses05F_impro04_F004     fru  1.5  3.5  4.0
...          ...       ...                     ...     ...  ...  ...  ...
2165      215.40    220.75  Ses05M_script03_1_M030     exc  4.5  4.5  3.5
2166      221.13    223.83  Ses05M_script03_1_M031     exc  2.5  3.5  3.5
2167      229.88    233.06  Ses05M_script03_1_M032     exc  4.5  4.5  4.5
2168      234.14    236.66  Ses05M_script03_1_M033     exc  4.0  3.0  3.5
2169      238.73    245.38  Ses05M_script03_1_M034     xxx  3.0  4.0  4.0

[2170 rows x 7 columns]


1it [00:03,  3.55s/it]

Some exception occured


2it [00:08,  3.97s/it]

Some exception occured


...

2169it [1:05:16,  1.43s/it]

Some exception occured


2170it [1:05:19,  1.84s/it]

Some exception occured


2170it [1:05:20,  1.81s/it]
